# Simulating lightcurves using a black body spectrum

This notebook shows an example of a simulating lightcurves with a blackbody SED for 90 days of ZTF observations. (The survey is restricted to 90 days in order to run reasonably fast. A similar simulation for a 3-year survey can be generated using `BlackBody_3yr.py` [TBA].)

In [1]:
import warnings
## No annoying warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import simsurvey.cadence as simul
import sncosmo

import simsurvey_tools as sst

In [2]:
# Set the time frame for the survey (90 days)
mjd_range = (58000, 58090)

The transient generator randomly draws all parameters needed to simulate the lightcurves. For this simulation, one can use `BlackBodySource` included in `simsurvey` to create a model. In addition, one needs to define a function the randomizes the parameters (temperature, peak magnitude and host $E(B-V)$ in this case) and a rate function. 

Further required input is the time frame, which should be set to include a number of days before and after the survey in order to simulate all SNe that may be visible during the observations. Additionally, to reduce the computation time, the  coordinate ranges can be restricted such that only few SNe outside the observed part of the sky are simulated.

In [3]:
# Need to define a basic lightcurve shape in arbitrary flux units
# Using eq. (1) of Bazin et al. (2011) for a basic shape
t_rise = 2.
t_fall = 10.
phase = np.linspace(-6*t_rise, 4*t_fall, 100)
flux = np.exp(-phase/t_fall)/(1 + np.exp(-phase/t_rise))

source = simul.BlackBodySource(phase, flux)

dust = sncosmo.CCM89Dust()
model = sncosmo.Model(source=source,
                      effects=[dust],
                      effect_names=['host'],
                      effect_frames=['rest'])

def random_parameters(redshifts, model,
                      mag=(-17.5, 0.5),
                      T=(2e4, 5e3),
                      r_v=2., ebv_rate=0.11,
                      **kwargs):
    temperatures = np.random.uniform(T[0]-T[1], T[0]+T[1], len(redshifts))
    
    # Amplitude
    amp = []
    for z, temp in zip(redshifts, temperatures):
        model.set(z=z, T=temp)
        mabs = np.random.normal(mag[0], mag[1])
        model.set_source_peakabsmag(mabs, 'bessellb', 'vega')
        amp.append(model.get('amplitude'))

    return {
        'amplitude': np.array(amp),
        'T': temperatures,
        'hostr_v': r_v * np.ones(len(redshifts)),
        'hostebv': np.random.exponential(ebv_rate, len(redshifts))
    }

transientprop = dict(lcmodel=model,
                    lcsimul_func=random_parameters,
                    lcsimul_prop=dict(mag=(-17.5, 0.5)))

tr = simul.get_transient_generator([0.0, 0.1], ratekind='custom',
                                   ratefunc=lambda z: 1e-6,
                                   dec_range=[-40,90],
                                   mjd_range=[mjd_range[0] - 90,
                                              mjd_range[1] + 25],
                                   transientprop=transientprop)

The plan object collects all required information of the survey strategy. The function used below generates a simple mock survey to get a basic idea of what we can expect from ZTF, see its docstring in `simsurvey_tools.py` for more information. The output of Eric's `ztf_sim` code can also be loaded into a plan object, see `plan_from_ztf_sim.ipynb`. (Currently the code uses DES $gri$ filters instead of the ZTF ones, which be included later.)

In [4]:
plan = sst.get_survey_plan_simple(mjd_range=mjd_range)

In [5]:
plan.cadence

band,skynoise,field,RA,time,Dec
str4,float64,int64,float64,float64,float64
desr,800.0,661,57.69565,58000.0,31.75385
desr,800.0,666,96.82609,58000.0005208,31.75385
desr,800.0,662,65.52174,58000.0010417,31.75385
desr,800.0,659,42.04348,58000.0015625,31.75385
desr,800.0,658,34.21739,58000.0020833,31.75385
desr,800.0,667,104.65217,58000.0026042,31.75385
desr,800.0,665,89.0,58000.003125,31.75385
desr,800.0,654,2.91304,58000.0036458,31.75385
desr,800.0,656,18.56522,58000.0041667,31.75385


Lastly, some basic properties of the instrument used in the survey must be defined but these are mostly chosen in order to get specific depths of the observations and need not be changed.

In [6]:
instprop = {"desg":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desr":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desi":{"gain":1.,"zp":30,"zpsys":'ab'}}

These three components are collected in a SimulSurvey object, which is then used to generate the lightcurves.

In [7]:
survey = simul.SimulSurvey(generator=tr, 
                           plan=plan, 
                           instprop=instprop)

lcs = survey.get_lightcurves(
    # This cell may give an error because of the progress bar widget.
    # Just delete the following line to fix this.
    progress_bar=True, notebook=True
)

idx = lcs.meta['idx_orig']
n_obs = np.zeros(survey.generator.ntransient)
n_obs[idx] = np.array([len(a) for a in lcs])

print 'SNe observed: %i out of %i'%(np.sum(n_obs > 0),
                                    survey.generator.ntransient)

Determining field IDs for all objects

Generating lightcurves

SNe observed: 13 out of 133


The output is a LightcurveCollection object that organizes the lightcurves and can be accessed as shown below.

In [8]:
lcs[0]

time,band,flux,fluxerr,zp,zpsys
float64,str4,float64,float64,int64,str2
58023.0536458,desr,1183.55184823,800.118027742,30,ab
58023.1369792,desr,745.246681024,800.121740594,30,ab
58024.0546875,desr,1202.06297378,800.17099876,30,ab
58024.1380208,desr,1008.66282301,800.176352049,30,ab
58025.0541667,desr,-576.508712569,800.247160822,30,ab
58025.1375,desr,135.92200388,800.254826281,30,ab
58026.0546875,desr,964.534005874,800.356324092,30,ab
58026.06875,desr,909.080095679,800.358149805,30,ab
58026.1380208,desr,1737.00776464,800.36726749,30,ab
